
<h1>Kubernetes</h1>



<img src="assets/k8s-logo.png" alt="k8s-logo" style="width: 400px;"/>

hallo

In [10]:
!git rm Untitled.ipynb

error: the following file has changes staged in the index:
    Untitled.ipynb
(use --cached to keep the file, or -f to force removal)


ll